In [1]:
import pycisTopic
import pickle
import pandas as pd


import os

import glob
import loompy as lp
import numpy as np

%load_ext lab_black

In [2]:
# Load class
class loom_file_creation:
    import os
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import scanpy as sc

    def __init__(self, dgem, meta):
        import numpy as np
        import pandas as pd

        self.dgem = dgem
        self.meta = meta
        self.CPM = self.dgem / self.dgem.sum() * 1e6
        self.logCPM = np.log1p(self.CPM)
        self.metaJson = {}
        self.metaJson["metrics"] = []
        self.metaJson["annotations"] = []
        # initialize counters
        self.default_embedding_counter = 0
        self.embeddings_id = 1
        self.clusterings_id = 0
        self.col_attrs_counter = 0
        self.row_attrs_counter = 0
        self.attrs_counter = 0
        # add other embeddings
        self.Embeddings_X = pd.DataFrame()
        self.Embeddings_Y = pd.DataFrame()
        # add clusterings
        self.clusterings = pd.DataFrame(index=self.meta.index)

    def add_default_embedding(self, x, y, name="default"):
        self.main_dr = self.meta[[x, y]]
        self.main_dr.columns = ["_X", "_Y"]
        self.metaJson["embeddings"] = [{"id": -1, "name": name}]
        self.default_embedding_counter = 1

    def add_other_embedding(self, x, y, name):
        # every extra embedding is put in these dataframes as columns
        self.Embeddings_X[str(self.embeddings_id)] = self.meta[x]
        self.Embeddings_Y[str(self.embeddings_id)] = self.meta[y]
        self.metaJson["embeddings"].append({"id": self.embeddings_id, "name": name})
        self.embeddings_id += 1

    # here you can add description of the cluster
    def add_numeric_clustering(self, res, group, name, annotated=None):
        # add all cluster IDs to meta
        if self.clusterings_id == 0:
            self.metaJson["clusterings"] = [
                {
                    "id": self.clusterings_id,
                    "group": group,
                    "name": name,
                    "clusters": [],
                }
            ]
        if self.clusterings_id > 0:
            # add all cluster IDs to meta
            self.metaJson["clusterings"].append(
                {
                    "id": self.clusterings_id,
                    "group": group,
                    "name": name,
                    "clusters": [],
                }
            )
        # get annotation for numeric values (count annotation for each numeric value, take the annotation that occurs the most)
        if annotated in self.meta.columns:
            annotation_dataframe = pd.DataFrame(
                meta.groupby(res)[annotated].value_counts().groupby(res).head(n=1)
            )
            annotation_dataframe.columns = ["test"]
            annotation_dataframe = annotation_dataframe.reset_index()
            annotation_dataframe.index = annotation_dataframe[res]
            annotation_dataframe.drop(res, axis=1, inplace=True)
            annotation_dataframe.drop("test", axis=1, inplace=True)
            for i in range(max(set([int(x) for x in self.meta[res]])) + 1):
                clustDict = {}
                clustDict["id"] = i
                clustDict["description"] = annotation_dataframe.loc[i, annotated]
                self.metaJson["clusterings"][self.clusterings_id]["clusters"].append(
                    clustDict
                )
        else:
            for i in range(max(set([int(x) for x in self.meta[res]])) + 1):
                clustDict = {}
                clustDict["id"] = i
                clustDict["description"] = f"Unannotated Cluster {i}"
                self.metaJson["clusterings"][self.clusterings_id]["clusters"].append(
                    clustDict
                )
        # add cluster IDs for every cell to dataframe
        self.clusterings[str(self.clusterings_id)] = self.meta[res].values.astype(
            np.int64
        )
        # increase counter
        self.clusterings_id += 1

    def dfToNamedMatrix(df):
        arr_ip = [tuple(i) for i in df.values]
        dtyp = np.dtype(list(zip(df.dtypes.index, df.dtypes)))
        arr = np.array(arr_ip, dtype=dtyp)
        return arr

    def make_col_attributes(self):
        # define col attributes
        if self.embeddings_id > 1:
            if self.clusterings_id > 0:
                self.col_attrs = {
                    "CellID": np.array(self.meta.index),
                    "Embedding": dfToNamedMatrix(self.main_dr),
                    "Embeddings_X": dfToNamedMatrix(self.Embeddings_X),
                    "Embeddings_Y": dfToNamedMatrix(self.Embeddings_Y),
                    "Clusterings": dfToNamedMatrix(self.clusterings),
                    "ClusterID": np.array(
                        self.clusterings[str(self.clusterings_id - 1)].values
                    ),
                }
            else:
                self.col_attrs = {
                    "CellID": np.array(self.meta.index),
                    "Embedding": dfToNamedMatrix(self.main_dr),
                    "Embeddings_X": dfToNamedMatrix(self.Embeddings_X),
                    "Embeddings_Y": dfToNamedMatrix(self.Embeddings_Y),
                }
        elif self.clusterings_id > 0:
            self.col_attrs = {
                "CellID": np.array(self.meta.index),
                "Embedding": dfToNamedMatrix(self.main_dr),
                "Clusterings": dfToNamedMatrix(self.clusterings),
                "ClusterID": np.array(
                    self.clusterings[str(self.clusterings_id - 1)].values
                ),
            }
        else:
            self.col_attrs = {
                "CellID": np.array(self.meta.index),
                "Embedding": dfToNamedMatrix(self.main_dr),
            }
        self.col_attrs_counter = 1

    def add_annotation(self, annotation):
        if self.col_attrs_counter < 1:
            print("first run make_col_attributes")
        self.metaJson["annotations"].append(
            {"name": annotation, "values": list(set(self.meta[annotation].values))}
        )
        self.col_attrs[annotation] = np.array(self.meta[annotation].values)

    def add_metric(self, metric):
        if self.col_attrs_counter < 1:
            print("first run make_col_attributes")
        self.metaJson["metrics"].append({"name": metric})
        self.col_attrs[metric] = np.array(self.meta[metric].values)

    def make_row_attributes(self):
        self.row_attrs = {"Gene": np.array(self.dgem.index)}
        self.row_attrs_counter = 1

    def make_attributes(self):
        import json
        import base64
        import zlib

        self.attrs = {"MetaData": json.dumps(self.metaJson)}
        self.attrs["MetaData"] = base64.b64encode(
            zlib.compress(json.dumps(self.metaJson).encode("ascii"))
        ).decode("ascii")
        self.attrs_counter = 1

    #     def add_attrs(self):
    #         import json
    #         import base64
    #         import zlib

    #         self.attrs = {"MetaData": json.dumps(self.metaJson)}
    #         self.attrs["MetaData"] = base64.b64encode(
    #             zlib.compress(json.dumps(self.metaJson).encode("ascii"))
    #         ).decode("ascii")
    #         self.attrs_counter = 1

    def create_loomfile(self, path):
        import loompy as lp

        if (
            self.col_attrs_counter == 1
            and self.row_attrs_counter == 1
            and self.default_embedding_counter == 1
            and self.attrs_counter == 1
        ):
            lp.create(
                filename=path,
                layers=(self.dgem).values,
                row_attrs=self.row_attrs,
                col_attrs=self.col_attrs,
                file_attrs=self.attrs,
            )
        else:
            print(
                "Make sure that col_attr, row_attrs, attrs and embedding have been run"
            )

In [3]:
def dfToNamedMatrix(df):
    arr_ip = [tuple(i) for i in df.values]
    dtyp = np.dtype(list(zip(df.dtypes.index, df.dtypes)))
    arr = np.array(arr_ip, dtype=dtyp)
    return arr

In [4]:
cto_path_dict = {
    x.split("/")[-1].split(".")[0] + ".FIXEDCELLS": x
    for x in sorted(
        glob.glob(
            "cistopic_objects/*.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.*.dimreduc.consensus.pkl"
        )
    )
}
cto_path_dict

{'BIO_ddseq_1.FIXEDCELLS': 'cistopic_objects/BIO_ddseq_1.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_11topics.dimreduc.consensus.pkl',
 'BIO_ddseq_2.FIXEDCELLS': 'cistopic_objects/BIO_ddseq_2.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_14topics.dimreduc.consensus.pkl',
 'BIO_ddseq_3.FIXEDCELLS': 'cistopic_objects/BIO_ddseq_3.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_10topics.dimreduc.consensus.pkl',
 'BIO_ddseq_4.FIXEDCELLS': 'cistopic_objects/BIO_ddseq_4.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_11topics.dimreduc.consensus.pkl',
 'BRO_mtscatac_1.FIXEDCELLS': 'cistopic_objects/BRO_mtscatac_1.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_11topics.dimreduc.consensus.pkl',
 'BRO_mtscatac_2.FIXEDCELLS': 'cistopic_objects/BRO_mtscatac_2.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_17topics.dimreduc.consensus.pkl',
 'CNA_10xmultiome_1.FIXEDCELLS': 'cistopic_objects/CNA_10xmultiome_1.FIXEDCELLS__cto.scrublet0-4.fmx.singlets.model_10topics.dimreduc.consensus.pkl',
 'CNA_10xm

In [5]:
import numpy as np

In [6]:
sex_dict = {"sampleA": "male", "sampleB": "female"}

In [7]:
for sample, path in cto_path_dict.items():
    print(sample)
    with open(path, "rb") as f:
        cto = pickle.load(f)

    df = pd.DataFrame(
        cto.fragment_matrix.toarray(), columns=cto.cell_names, index=cto.region_names
    )

    loomfile = loom_file_creation(df, cto.cell_data)
    cto.cell_data["tSNE_X"] = cto.projections["cell"]["tSNE"]["tSNE_1"]
    cto.cell_data["tSNE_Y"] = cto.projections["cell"]["tSNE"]["tSNE_2"]
    # cto.cell_data["harmony_tSNE_X"] = cto.projections["cell"]["tSNE_harmony"]["tSNE_1"]
    # cto.cell_data["harmony_tSNE_Y"] = cto.projections["cell"]["tSNE_harmony"]["tSNE_2"]
    # loomfile.add_default_embedding(x="harmony_tSNE_X", y="harmony_tSNE_Y")
    loomfile.add_default_embedding(x="tSNE_X", y="tSNE_Y")
    # loomfile.add_numeric_clustering(res="pycisTopic_leiden_10_0.9",group="pycisTopic_leiden_10_0.9",name="pycisTopic_leiden_10_0.9",annotated='cell_type')
    # loomfile.add_numeric_clustering(res="integrated_snn_res.2",group="Louvain",name="Unannotated")
    loomfile.make_col_attributes()
    loomfile.make_row_attributes()
    # loomfile.add_annotation(annotation="seurat_cell_type_full_mega_harmony_consensus")
    loomfile.add_annotation(annotation="seurat_cell_type")
    loomfile.add_annotation(annotation="pycisTopic_leiden_10_3.0")
    if "fmx_sample" in cto.cell_data.columns:
        cto.cell_data["sex"] = [sex_dict[x] for x in cto.cell_data["fmx_sample"]]
        loomfile.add_annotation(annotation="fmx_sample")

    loomfile.add_metric(metric="Unique_nr_frag_in_regions")
    loomfile.add_metric(metric="Dupl_rate")
    loomfile.add_metric(metric="FRIP")
    loomfile.add_metric(metric="TSS_enrichment")
    # loomfile.add_metric(metric="seurat_cell_type_full_mega_pred_score")
    loomfile.add_metric(metric="seurat_cell_type_pred_score")
    cto.cell_data["Scrublet_doublet_score"] = cto.cell_data["Doublet_scores_fragments"]
    loomfile.add_metric(metric="Scrublet_doublet_score")
    # loomfile.add_metric(metric="n_genes")
    loomfile.make_attributes()
    loomfile.create_loomfile(f"scope_looms/{sample}__count_matrix_in_own_peaks.loom")

BIO_ddseq_1.FIXEDCELLS
BIO_ddseq_2.FIXEDCELLS
BIO_ddseq_3.FIXEDCELLS
BIO_ddseq_4.FIXEDCELLS
BRO_mtscatac_1.FIXEDCELLS
BRO_mtscatac_2.FIXEDCELLS
CNA_10xmultiome_1.FIXEDCELLS
CNA_10xmultiome_2.FIXEDCELLS
CNA_10xv11_1.FIXEDCELLS
CNA_10xv11_2.FIXEDCELLS
CNA_10xv11_3.FIXEDCELLS
CNA_10xv11_4.FIXEDCELLS
CNA_10xv11_5.FIXEDCELLS
CNA_10xv2_1.FIXEDCELLS
CNA_10xv2_2.FIXEDCELLS
CNA_hydrop_1.FIXEDCELLS
CNA_hydrop_2.FIXEDCELLS
CNA_hydrop_3.FIXEDCELLS
CNA_mtscatac_1.FIXEDCELLS
CNA_mtscatac_2.FIXEDCELLS
EPF_hydrop_1.FIXEDCELLS
EPF_hydrop_2.FIXEDCELLS
EPF_hydrop_3.FIXEDCELLS
EPF_hydrop_4.FIXEDCELLS
HAR_ddseq_1.FIXEDCELLS
HAR_ddseq_2.FIXEDCELLS
MDC_mtscatac_1.FIXEDCELLS
MDC_mtscatac_2.FIXEDCELLS
OHS_s3atac_1.FIXEDCELLS
OHS_s3atac_2.FIXEDCELLS
SAN_10xmultiome_1.FIXEDCELLS
SAN_10xmultiome_2.FIXEDCELLS
STA_10xv11_1.FIXEDCELLS
STA_10xv11_2.FIXEDCELLS
TXG_10xv11_1.FIXEDCELLS
TXG_10xv2_1.FIXEDCELLS
TXG_10xv2_2.FIXEDCELLS
UCS_ddseq_1.FIXEDCELLS
UCS_ddseq_2.FIXEDCELLS
VIB_10xmultiome_1.FIXEDCELLS
VIB_10xmultiome

In [9]:
for loom_file in glob.glob("scope_looms/*.loom"):
    print(loom_file)
    with lp.connect(loom_file, "r+", validate=False) as ds:
        ds.attrs["SCopeTreeL1"] = "scATAC_seq"
        ds.attrs["SCopeTreeL2"] = "Downsampled_data_merged"
        ds.attrs["SCopeTreeL3"] = "Count_matrix_in_sample_specific_peaks"

scope_looms/VIB_10xv2_1.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/TXG_10xv2_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/CNA_10xv11_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/VIB_hydrop_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/BIO_ddseq_4.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/CNA_hydrop_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/VIB_10xv1_1.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/CNA_10xv2_1.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/BIO_ddseq_1.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/EPF_hydrop_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/VIB_10xmultiome_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/CNA_10xmultiome_2.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/UCS_ddseq_1.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/BIO_ddseq_3.FIXEDCELLS__count_matrix_in_own_peaks.loom
scope_looms/VIB_hydrop_1.FIXEDCELLS__count_mat